In [28]:
!pip install gymnasium
!pip install "gymnasium[atari, accept-rom-license]"
!apt-get install -y swig
!pip install gymnasium[box2d]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
swig is already the newest version (4.0.2-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [29]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import Variable
from collections import deque, namedtuple

In [30]:
class Network(nn.Module):

  def __init__(self, state_size, action_size, seed = 42):
    super(Network, self).__init__()
    self.seed = torch.manual_seed(seed)
    self.fc1 = nn.Linear(state_size, 64)
    self.fc2 = nn.Linear(64, 64)
    self.fc3 = nn.Linear(64, action_size)

  def forward(self, state):
    x = self.fc1(state)
    x = F.relu(x)
    x = self.fc2(x)
    x = F.relu(x)
    return self.fc3(x)

In [31]:
import gymnasium as gym
env = gym.make('LunarLander-v3')
state_shape = env.observation_space.shape
state_size = env.observation_space.shape[0]
number_actions = env.action_space.n
print('State shape: ', state_shape)
print('State size: ', state_size)
print('Number of actions: ', number_actions)

State shape:  (8,)
State size:  8
Number of actions:  4


In [32]:
class ReplayMemory(object):

  def __init__(self, capacity):
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.capacity = capacity
    self.memory = []

  def push(self, event):
    self.memory.append(event)
    if len(self.memory) > self.capacity:
      del self.memory[0]

  def sample(self, batch_size):
    experiences = random.sample(self.memory, k = batch_size)
    states = torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None])).float().to(self.device)
    actions = torch.from_numpy(np.vstack([e[1] for e in experiences if e is not None])).long().to(self.device)
    rewards = torch.from_numpy(np.vstack([e[2] for e in experiences if e is not None])).float().to(self.device)
    next_states = torch.from_numpy(np.vstack([e[3] for e in experiences if e is not None])).float().to(self.device)
    dones = torch.from_numpy(np.vstack([e[4] for e in experiences if e is not None]).astype(np.uint8)).float().to(self.device)
    return states, next_states, actions, rewards, dones

In [33]:
class Agent():

  def __init__(self, state_size, action_size, learning_rate, minibatch_size, discount_factor, replay_buffer_size, interpolation_parameter ):
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.state_size = state_size
    self.action_size = action_size
    self.local_qnetwork = Network(state_size, action_size).to(self.device)
    self.target_qnetwork = Network(state_size, action_size).to(self.device)
    self.optimizer = optim.Adam(self.local_qnetwork.parameters(), lr = learning_rate)
    self.memory = ReplayMemory(replay_buffer_size)
    self.t_step = 0

  def step(self, state, action, reward, next_state, done):
    self.memory.push((state, action, reward, next_state, done))
    self.t_step = (self.t_step + 1) % 4
    if self.t_step == 0:
      if len(self.memory.memory) > minibatch_size:
        experiences = self.memory.sample(100)
        self.learn(experiences, discount_factor)

  def load_model(self, file_path):
        self.local_qnetwork.load_state_dict(torch.load(file_path, map_location=self.device))
        self.local_qnetwork.eval()

  def act(self, state, epsilon = 0.):
    state = torch.from_numpy(state).float().unsqueeze(0).to(self.device)
    self.local_qnetwork.eval()
    with torch.no_grad():
      action_values = self.local_qnetwork(state)
    self.local_qnetwork.train()
    if random.random() > epsilon:
      return np.argmax(action_values.cpu().data.numpy())
    else:
      return random.choice(np.arange(self.action_size))

  def learn(self, experiences, discount_factor):
    states, next_states, actions, rewards, dones = experiences
    next_q_targets = self.target_qnetwork(next_states).detach().max(1)[0].unsqueeze(1)
    q_targets = rewards + discount_factor * next_q_targets * (1 - dones)
    q_expected = self.local_qnetwork(states).gather(1, actions)
    loss = F.mse_loss(q_expected, q_targets)
    self.optimizer.zero_grad()
    loss.backward()
    self.optimizer.step()
    self.soft_update(self.local_qnetwork, self.target_qnetwork, interpolation_parameter)

  def soft_update(self, local_model, target_model, interpolation_parameter):
    for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
      target_param.data.copy_(interpolation_parameter * local_param.data + (1.0 - interpolation_parameter) * target_param.data)

In [ ]:
'''
agent = Agent(state_size, number_actions)
learning_rate = 5e-4
minibatch_size = 100
discount_factor = 0.99
replay_buffer_size = int(1e5)
interpolation_parameter = 1e-3
'''

In [ ]:
from itertools import product


# Define hyperparameter ranges
learning_rates = [1e-4, 5e-4, 1e-3]
minibatch_sizes = [64, 100, 128]
discount_factors = [0.95, 0.97, 0.99]
buffer_sizes = [int(1e4), int(1e5), int(5e5)]
interpolation_parameters = [1e-4, 1e-3, 5e-3]

# Create all combinations
combinations = list(product(learning_rates, minibatch_sizes, discount_factors, buffer_sizes, interpolation_parameters))

# Placeholder for actual DQN training function
def train_dqn(lr, batch_size, gamma, buffer_size, tau):
    print(f"\nTraining with: LR={lr}, Batch={batch_size}, Gamma={gamma}, Buffer={buffer_size}, Tau={tau}")
    agent = Agent(state_size, number_actions, lr, batch_size, gamma, buffer_size, tau)
    number_episodes = 2000
    maximum_number_timesteps_per_episode = 1000
    epsilon_starting_value  = 1.0
    epsilon_ending_value  = 0.01
    epsilon_decay_value  = 0.995
    epsilon = epsilon_starting_value
    scores_on_50_episodes = deque(maxlen = 50)

    avg_score=0
    for episode in range(1, 51):
      state, _ = env.reset()
      score = 0
      for t in range(maximum_number_timesteps_per_episode):
        action = agent.act(state, epsilon)
        next_state, reward, done, _, _ = env.step(action)
        agent.step(state, action, reward, next_state, done)
        state = next_state
        score += reward
        if done:
          break
      scores_on_50_episodes.append(score)
      epsilon = max(epsilon_ending_value, epsilon_decay_value * epsilon)
      print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_50_episodes)), end = "")
      if episode % 50 == 0:
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_50_episodes)))
        avg_score = np.mean(scores_on_50_episodes)
        break

    return avg_score

# Run grid search
results = []
for i, (lr, bs, gamma, buf, tau) in enumerate(combinations):
    print(f"\n>>> [{i+1}/{len(combinations)}] Hyperparameter set")
    score = train_dqn(lr, bs, gamma, buf, tau)
    results.append(((lr, bs, gamma, buf, tau), score))

# Sort by score descending
results.sort(key=lambda x: x[1], reverse=True)

# Show top 5 results
print("\nTop 5 configurations:")
for combo, score in results[:5]:
    print(f"Params: {combo}, Score: {score}")





>>> [1/243] Hyperparameter set

Training with: LR=0.0001, Batch=64, Gamma=0.95, Buffer=10000, Tau=0.0001
Episode 50	Average Score: -213.10

>>> [2/243] Hyperparameter set

Training with: LR=0.0001, Batch=64, Gamma=0.95, Buffer=10000, Tau=0.001
Episode 50	Average Score: -196.17

>>> [3/243] Hyperparameter set

Training with: LR=0.0001, Batch=64, Gamma=0.95, Buffer=10000, Tau=0.005
Episode 50	Average Score: -196.27

>>> [4/243] Hyperparameter set

Training with: LR=0.0001, Batch=64, Gamma=0.95, Buffer=100000, Tau=0.0001
Episode 50	Average Score: -189.29

>>> [5/243] Hyperparameter set

Training with: LR=0.0001, Batch=64, Gamma=0.95, Buffer=100000, Tau=0.001
Episode 50	Average Score: -245.83

>>> [6/243] Hyperparameter set

Training with: LR=0.0001, Batch=64, Gamma=0.95, Buffer=100000, Tau=0.005
Episode 50	Average Score: -195.83

>>> [7/243] Hyperparameter set

Training with: LR=0.0001, Batch=64, Gamma=0.95, Buffer=500000, Tau=0.0001
Episode 50	Average Score: -189.69

>>> [8/243] Hyperpa

In [ ]:
print("Best params: ", results[0][0])


Best params:  (0.0005, 64, 0.99, 100000, 0.005)


In [ ]:
from itertools import product


# Define hyperparameter ranges
learning_rates = [1e-4, 5e-4, 1e-3]
discount_factors = [0.95, 0.97, 0.99]
buffer_sizes = [int(1e4), int(1e5), int(5e5)]
interpolation_parameters = [1e-4, 1e-3, 5e-3]

# Create all combinations
combinations = list(product(learning_rates, discount_factors, buffer_sizes, interpolation_parameters))

# Placeholder for actual DQN training function
def train_dqn(lr, batch_size, gamma, buffer_size, tau):
    print(f"\nTraining with: LR={lr}, Gamma={gamma}, Buffer={buffer_size}, Tau={tau}")
    agent = Agent(state_size, number_actions, lr, 100, gamma, buffer_size, tau)
    number_episodes = 2000
    maximum_number_timesteps_per_episode = 1000
    epsilon_starting_value  = 1.0
    epsilon_ending_value  = 0.01
    epsilon_decay_value  = 0.995
    epsilon = epsilon_starting_value
    scores_on_50_episodes = deque(maxlen = 50)

    avg_score=0
    for episode in range(1, 51):
      state, _ = env.reset()
      score = 0
      for t in range(maximum_number_timesteps_per_episode):
        action = agent.act(state, epsilon)
        next_state, reward, done, _, _ = env.step(action)
        agent.step(state, action, reward, next_state, done)
        state = next_state
        score += reward
        if done:
          break
      scores_on_50_episodes.append(score)
      epsilon = max(epsilon_ending_value, epsilon_decay_value * epsilon)
      print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_50_episodes)), end = "")
      if episode % 50 == 0:
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_50_episodes)))
        avg_score = np.mean(scores_on_50_episodes)
        break

    return avg_score

# Run grid search
results = []
for i, (lr, gamma, buf, tau) in enumerate(combinations):
    print(f"\n>>> [{i+1}/{len(combinations)}] Hyperparameter set")
    score = train_dqn(lr, 100, gamma, buf, tau)
    results.append(((lr, 100, gamma, buf, tau), score))

# Sort by score descending
results.sort(key=lambda x: x[1], reverse=True)

# Show top 5 results
print("\nTop 5 configurations:")
for combo, score in results[:5]:
    print(f"Params: {combo}, Score: {score}")





>>> [1/81] Hyperparameter set

Training with: LR=0.0001, Gamma=0.95, Buffer=10000, Tau=0.0001
Episode 50	Average Score: -222.19

>>> [2/81] Hyperparameter set

Training with: LR=0.0001, Gamma=0.95, Buffer=10000, Tau=0.001
Episode 50	Average Score: -188.54

>>> [3/81] Hyperparameter set

Training with: LR=0.0001, Gamma=0.95, Buffer=10000, Tau=0.005
Episode 50	Average Score: -218.02

>>> [4/81] Hyperparameter set

Training with: LR=0.0001, Gamma=0.95, Buffer=100000, Tau=0.0001
Episode 50	Average Score: -181.02

>>> [5/81] Hyperparameter set

Training with: LR=0.0001, Gamma=0.95, Buffer=100000, Tau=0.001
Episode 50	Average Score: -199.48

>>> [6/81] Hyperparameter set

Training with: LR=0.0001, Gamma=0.95, Buffer=100000, Tau=0.005
Episode 50	Average Score: -223.26

>>> [7/81] Hyperparameter set

Training with: LR=0.0001, Gamma=0.95, Buffer=500000, Tau=0.0001
Episode 50	Average Score: -194.59

>>> [8/81] Hyperparameter set

Training with: LR=0.0001, Gamma=0.95, Buffer=500000, Tau=0.001
Ep

In [ ]:
print("Best params: ", results[0][0])


Best params:  (0.001, 100, 0.97, 500000, 0.001)


In [34]:
learning_rate = 5e-4
minibatch_size = 100
discount_factor = 0.99
replay_buffer_size = int(1e5)
interpolation_parameter = 5e-3
agent = Agent(state_size, number_actions, learning_rate, minibatch_size, discount_factor, replay_buffer_size, interpolation_parameter)
print(learning_rate, minibatch_size, discount_factor, replay_buffer_size, interpolation_parameter)

0.0005 100 0.99 100000 0.005


In [35]:
number_episodes = 2000
maximum_number_timesteps_per_episode = 1000
epsilon_starting_value  = 1.0
epsilon_ending_value  = 0.01
epsilon_decay_value  = 0.995
epsilon = epsilon_starting_value
scores_on_100_episodes = deque(maxlen = 64)

for episode in range(1, number_episodes + 1):
  state, _ = env.reset()
  score = 0
  for t in range(maximum_number_timesteps_per_episode):
    action = agent.act(state, epsilon)
    next_state, reward, done, _, _ = env.step(action)
    agent.step(state, action, reward, next_state, done)
    state = next_state
    score += reward
    if done:
      break
  scores_on_100_episodes.append(score)
  epsilon = max(epsilon_ending_value, epsilon_decay_value * epsilon)
  print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)), end = "")
  if episode % 64 == 0:
    print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)))
  if np.mean(scores_on_100_episodes) >= 200.0:
    print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(episode - 64, np.mean(scores_on_100_episodes)))
    torch.save(agent.local_qnetwork.state_dict(), 'checkpoint_best_params.pth')
    break

Episode 64	Average Score: -167.25
Episode 128	Average Score: -82.97
Episode 192	Average Score: -66.58
Episode 256	Average Score: -39.17
Episode 320	Average Score: 2.20
Episode 384	Average Score: 203.86

Environment solved in 320 episodes!	Average Score: 203.86


In [ ]:
agent.load_model("checkpoint.pth")

In [36]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display
from gym.wrappers.monitoring.video_recorder import VideoRecorder

def show_video_of_model(agent, env_name):
    env = gym.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frames = []
    while not done:
        frame = env.render()
        frames.append(frame)
        action = agent.act(state)
        state, reward, done, _, _ = env.step(action.item())
    env.close()
    imageio.mimsave('video.mp4', frames, fps=30)

show_video_of_model(agent, 'LunarLander-v3')

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()